In [74]:
import pandas as pd
import numpy as np
import nltk
import re
from textblob import TextBlob
from gensim import corpora, models, similarities

In [75]:
dados = pd.read_csv('../lab_01/data/estadao_noticias_eleicao.csv')
dados = dados.replace(np.nan, '', regex=True)

In [76]:
dados.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


In [77]:
print("Os dados tem %d observações e %d colunas" % dados.shape)

Os dados tem 8716 observações e 6 colunas


In [78]:
dados["text"] = dados["titulo"] + " " + dados["subTitulo"] + " " + dados["conteudo"]

In [79]:
dados.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia,text
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1,PT espera 30 mil pessoas em festa na Esplanada...
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2,Alckmin toma posse de olho no Planalto Governa...
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3,Seis obstáculos e desafios do segundo mandato ...
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4,Veja as principais fotos do dia e dos eventos...
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5,Veja as principais fotos do dia e dos eventos...


In [80]:
def remove_stopwords (text):
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    words = [i for i in text.split() if not i in stopwords]
    return (" ".join(words))

def keep_letters_only(text):
    return re.sub(r'[0-9]+', " ", text)

def standardize_text(text):
    text = text.lower()
    text = remove_stopwords(text)
    text = keep_letters_only(text)
    return text

dados.text = dados.text.apply(standardize_text)

In [81]:
dados.head()["text"]

0    pt espera   mil pessoas festa esplanada objeti...
1    alckmin toma posse olho planalto governador re...
2    seis obstáculos desafios segundo mandato dilma...
3    veja principais fotos dia eventos especiais br...
4    veja principais fotos dia eventos especiais br...
Name: text, dtype: object

In [82]:
def get_palavras(data):
    palavras = []
    for index, row in data.iterrows():
        row["text"] = row["text"].lower()
        palavras_doc = TextBlob(row["text"]).words
        palavras.append(palavras_doc)
    return palavras

In [83]:
texts = get_palavras(dados)

In [84]:
# remove words that appear less then 10 times
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 10] for text in texts]

In [85]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(14245 unique tokens: ['a', 'abrigo', 'adversários', 'advogado', 'afirma']...)


In [86]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [87]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=1000)

In [88]:
index = similarities.MatrixSimilarity(lsi[corpus]) # transform corpus to LSI space and index it

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [89]:
doc = "petrobrás"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
# print(vec_lsi)

In [90]:
sims = index[vec_lsi]
sims = sorted(enumerate(sims), key=lambda item: -item[1]) # perform a similarity query against the corpus
print(sims[0:3])

[(6592, 0.5940671), (2339, 0.49798048), (6764, 0.4927531)]


In [91]:
for i in range(0,3):
    print(dictionary[sims[i][0]])

recife
intuito
dirigida


In [92]:
def busca_palavras_similares(palavra):
    vec_bow = dictionary.doc2bow(palavra.lower().split())
    vec_lsi = lsi[vec_bow] # convert the query to LSI space
    sims = index[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1]) # perform a similarity query against the corpus
    
    return [dictionary[sims[0][0]], dictionary[sims[1][0]], dictionary[sims[2][0]]]
    

In [93]:
print(busca_palavras_similares("estado"))

['sbm', 'avaliação', 'sério']


## Realizando Buscas

In [94]:
def calcula_indices_invertidos(data):
    indices = {}
    for index, row in data.iterrows():
        palavras = TextBlob(row["text"]).words
        
        for palavra in palavras:
            palavra_low = palavra.lower()
            if(palavra_low in indices):
                indices[palavra_low].add(row["idNoticia"])
            else:
                indices[palavra_low] = {row["idNoticia"]}
                
    return(indices)

indices_invertidos = calcula_indices_invertidos(dados)

In [95]:
def busca_um_termo(termo):
    try:
        return(indices_invertidos[termo])
    except KeyError:
        return(set())

def busca_or(termo1, termo2, termo3, termo4):
    try:
        busca1 = indices_invertidos[termo1]
    except KeyError:
        busca1 = set()
        
    try:
        busca2 = indices_invertidos[termo2]
    except KeyError:
        busca2 = set()
        
    try:
        busca3 = indices_invertidos[termo3]
    except KeyError:
        busca3 = set()
    
    try:
        busca4 = indices_invertidos[termo4]
    except KeyError:
        busca4 = set()
        
    return(busca1 | busca2 | busca3 | busca4)

In [96]:
def busca(*args):
    if (len(args) != 4 and len(args) != 1):
        return("Número inválido de argumentos")
    elif(len(args) == 1):
        termo = args[0].lower()
        return(busca_um_termo(termo))
    elif(len(args) == 4):
        termo1 = args[0].lower()
        termo2 = args[1].lower()
        termo3 = args[2].lower()
        termo4 = args[3].lower()
        return(busca_or(termo1, termo2, termo3, termo4))

In [108]:
def busca_word2vec(palavra):
    print("A busca pelo termo %s apenas retorna:" % palavra)
    result_busca_palavra = busca(palavra)
    print("%d resultados, que são os documentos:" % len(result_busca_palavra))
    print(result_busca_palavra)
    t1, t2, t3 = busca_palavras_similares(palavra)
    print("Usando o modelo word2vec a consulta é extendida para os termos %s, %s e %s" % (t1, t2, t3))
    result_busca_extendida = busca(palavra, t1, t2, t3)
    print("Resultando nos seguintes %d documentos:" % len(result_busca_extendida))
    print(result_busca_extendida)
    print("Foram adicionados %d documentos ao retorno da consulta" % (len(result_busca_extendida) - len(result_busca_palavra)))

In [109]:
busca_word2vec("brasil")

A busca pelo termo brasil apenas retorna:
3223 resultados, que são os documentos:
{4, 5, 6, 7, 8, 8196, 10, 12, 13, 14, 15, 16, 8205, 18, 8208, 20, 8209, 8210, 23, 8213, 8214, 8217, 27, 28, 8221, 31, 32, 35, 37, 38, 8229, 41, 42, 43, 44, 8233, 46, 8236, 48, 8238, 50, 8240, 8241, 8242, 8245, 8246, 8247, 8248, 8249, 8253, 8254, 63, 8255, 65, 66, 67, 70, 72, 74, 76, 77, 8272, 84, 86, 8286, 95, 8287, 97, 8292, 101, 102, 107, 108, 8299, 8300, 8301, 8302, 8303, 114, 115, 8304, 117, 118, 8306, 8312, 121, 8313, 123, 8318, 128, 130, 133, 135, 136, 8328, 138, 140, 141, 143, 144, 8335, 8336, 147, 8338, 8339, 150, 8343, 152, 153, 155, 157, 8350, 8354, 163, 167, 168, 8361, 8362, 173, 174, 175, 176, 177, 8370, 8371, 8372, 184, 8376, 186, 8380, 189, 191, 192, 8385, 8386, 8387, 196, 8391, 200, 8393, 202, 203, 204, 205, 8397, 207, 8399, 8402, 211, 214, 8408, 218, 222, 224, 225, 226, 227, 8416, 230, 231, 8422, 233, 8428, 8429, 238, 8430, 240, 244, 8437, 246, 247, 248, 8442, 253, 254, 255, 256, 8450, 259